# 循环神经网络的从零开始实现

在本节中，我们将从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上训练一个模型来进行歌词创作。首先，我们读取周杰伦专辑歌词数据集：

In [1]:
import d2ltorch as d2lt
import math
import torch
from torch import nn
import time

(corpus_indices, char_to_idx, idx_to_char, 
 vocab_size) = d2lt.load_data_jay_lyrics()

## one-hot向量

为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。假设词典中不同字符的数量为$N$（即词典大小`vocab_size`），每个字符已经同一个从0到$N-1$的连续整数值索引一一对应。如果一个字符的索引是整数$i$, 那么我们创建一个全0的长为$N$的向量，并将其位置为$i$的元素设成1。该向量就是对原字符的one-hot向量。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [2]:
def one_hot(idx, size, device='cpu'): # 本函数已保存在d2ltorch包中方便以后使用
    batch_size = idx.size(0)
    index = idx.reshape(-1, 1)
    return torch.zeros(batch_size, size).to(device).scatter_(dim=1, index=index, value=1)

one_hot(torch.LongTensor([0, 2]), vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

我们每次采样的小批量的形状是(批量大小, 时间步数)。下面的函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵，矩阵个数等于时间步数。也就是说，时间步$t$的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中$n$为批量大小，$d$为输入个数，即one-hot向量长度（词典大小）。

In [3]:
def to_onehot(X, size, device='cpu'): # 本函数已保存在d2ltorch包中方便以后使用
    return [one_hot(x, size, device) for x in X.long().t()]

X = torch.arange(10).reshape(2, 5)
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, torch.Size([2, 1027]))

## 初始化模型参数

接下来，我们初始化模型参数。隐藏单元个数 `num_hiddens`是一个超参数。

In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
device = d2lt.try_gpu()
print('will use', device)

def get_params():
    def _one(shape):
        return torch.normal(mean=torch.zeros(shape), std=0.01).to(device)
    
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens).to(device)
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs).to(device)
    # 附上梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    
    for param in params:
        param.requires_grad = True
    return params

will use cuda:0


## 定义模型

我们根据循环神经网络的计算表达式实现该模型。首先定义`init_rnn_state`函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的`Tensor`组成的元组。使用元组是为了更便于处理隐藏状态含有多个`Tensor`的情况。

In [5]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros(batch_size, num_hiddens).to(device), )

下面的`rnn`函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了tanh函数。[“多层感知机”](../chapter_deep-learning-basics/mlp.ipynb)一节中介绍过，当元素在实数域上均匀分布时，tanh函数值的均值为0。

In [6]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(X.mm(W_xh) + H.mm(W_hh) + b_h)
        Y = H.mm(W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出的形状和隐藏状态的形状。

In [7]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size, device)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, torch.Size([2, 1027]), torch.Size([2, 256]))

## 定义预测函数

以下函数基于前缀`prefix`（含有数个字符的字符串）来预测接下来的`num_chars`个字符。这个函数稍显复杂，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [8]:
# 本函数已保存在d2ltorch包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    with torch.no_grad():
        for t in range(num_chars + len(prefix) - 1):
            # 将上一时间片的输出作为当前时间步的输入
            X = to_onehot(torch.Tensor([[output[-1]]]).to(device), vocab_size, device)
            # 计算输出和更新隐藏状态
            (Y, state) = rnn(X, state, params)
            # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
            if t < len(prefix) - 1:
                output.append(char_to_idx[prefix[t + 1]])
            else:
                output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

我们先测试一下`predict_rnn`函数。我们将根据前缀“分开”创作长度为10个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [9]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
           device, idx_to_char, char_to_idx)

'分开恩病古打翔沟朵危丘办'

## 裁剪梯度

循环神经网络中较容易出现梯度衰减或梯度爆炸。我们会在[“通过时间反向传播”](bptt.ipynb)一节中解释原因。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度

$$ \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}$$

的$L_2$范数不超过$\theta$。

In [10]:
# 本函数已保存在d2ltorch包中方便以后使用
def grad_clipping(params, theta, device):
    norm = torch.Tensor([0]).to(device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data.mul_(theta / norm)

## 困惑度

我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下[“softmax回归”](../chapter_deep-learning-basics/softmax-regression.ipynb)一节中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`。

## 定义模型训练函数

跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

1. 使用困惑度评价模型。
2. 在迭代模型参数前裁剪梯度。
3. 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关讨论可参考[“语言模型数据集（周杰伦专辑歌词）”](lang-model-dataset.ipynb)一节。

另外，考虑到后面将介绍的其他循环神经网络，为了更通用，这里的函数实现更长一些。

In [11]:
# 本函数已保存在d2ltorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2lt.data_iter_random
    else:
        data_iter_fn = d2lt.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter: # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
 
            inputs = to_onehot(X, vocab_size, device)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch_size * num_steps 的向量，这样跟输出的行一一对应
            y = Y.t().flatten().long()
            
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2lt.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.data.item() * torch.numel(y.data)
            n += torch.numel(y.data)
            
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## 训练模型并创作歌词

现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀“分开”和“不分开”分别创作长度为50个字符（不考虑前缀长度）的一段歌词。我们每过50个迭代周期便根据当前训练的模型创作一段歌词。

In [12]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

下面采用随机采样训练模型并创作歌词。

In [13]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 71.102366, time 0.30 sec
 - 分开 我想想这 我 想这你 别子的美 全有了这 我颗我这 我想想这 我想想这 我有我这 我想想这 我有我
 - 不分开 快颗去我 你的手空 我想我这多  后 我有你的可爱 我有你这 我想想这 我有我这 我想想这 我有我
epoch 100, perplexity 10.258602, time 0.30 sec
 - 分开 有使用 一步两 三颗四 干什么 干什么 干什么 干什么 干什么 干什么 干什么 干什么 干什么 干
 - 不分开柳 我不要再想你 不知不觉 你已经 不么我 娘子就 干什么 干什么 一九四 一颗的让我心红的可爱女人
epoch 150, perplexity 2.849834, time 0.30 sec
 - 分开 不想了你 全成之空 我一定纵 恨真的让 你已放头 我让后知不要再 牵着你 手 我对儿有 不亮我不见
 - 不分开吗 我叫好爸 你知我妈 这样的从蜜 用制茶 一果空 装属我抬脚这开 但使用  我的 爸有时间 全小放
epoch 200, perplexity 1.603830, time 0.29 sec
 - 分开 一直会停留 谁让它 一句惹毛我的人有危险 一再重演千年的誓言 一切又重演 祭司 神殿 征战 弓箭 
 - 不分开期 我叫你爸 你打我妈 这样对空 你一种空 说我不懂 说了没用 他的感容 还一秒钟 不敢不同 你已懂
epoch 250, perplexity 1.299777, time 0.32 sec
 - 分开 一直令你 心成了容  一直 废不风不 快使用双截棍 哼哼哈兮 快使用双截棍 哼 我用手刀防飞  说
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 它想我回见 让静我习著够  你去


接下来采用相邻采样训练模型并创作歌词。

In [14]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 63.158961, time 0.28 sec
 - 分开 我想要这 你谁我有 不要我有 不要我有 不天我有 不要我有 不要我有 不要我有 不天我有 不要我有
 - 不分开 我不要这 不果我有 快谁我有 不要我有 快谁我有 不要我有 不天我有 不要我有 不要我有 不天我有
epoch 100, perplexity 7.104511, time 0.30 sec
 - 分开 我说要这生活 我该你好生活 不知不觉 快天用沙凉过 一北的客栈人多 牧草有没有 我马儿有些瘦 我不
 - 不分开难爱 我想要你的微笑有天都能看到  我知道这里很美但家乡的你更美 刻能来暴 在谁的脚 在人己梦 你一
epoch 150, perplexity 2.055035, time 0.29 sec
 - 分开 这说的话去还水 是一枝人 温亮是有 在红海中 你在没空 恨我不懂 在我没中 在了没真 你不放 连一
 - 不分开觉 就已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.277912, time 0.30 sec
 - 分开 我说能 生九走睛年着 有话人看着我 抛物线进球 单手过人运球 篮下妙传出手 漂亮的假动作 帅呆了我
 - 不分开觉远就多 我想有没有坦堡 就像是童话故事  有教堂有城堡 每天忙碌地的寻找 到底什么我想要 却发现迷
epoch 250, perplexity 1.147400, time 0.30 sec
 - 分开 问候是人子的慢白 古堡金酒三点阳光 他们儿子我习惯 从小现迷了路怎么 也是开了了口斯B难过 印地安
 - 不分开觉远你多 我想你没斯坦着道 什么我都做得到 但那个人已经不是我 上海一九四三 泛黄的春联还残 我来有


## 小结

* 可以用基于字符级循环神经网络的语言模型来生成文本序列，例如创作歌词。
* 当训练循环神经网络时，为了应对梯度爆炸，可以裁剪梯度。
* 困惑度是对交叉熵损失函数做指数运算后得到的值。


## 练习

* 调调超参数，观察并分析对运行时间、困惑度以及创作歌词的结果造成的影响。
* 不裁剪梯度，运行本节中的代码，结果会怎样？
* 将`pred_period`变量设为1，观察未充分训练的模型（困惑度高）是如何创作歌词的。你获得了什么启发？
* 将相邻采样改为不从计算图分离隐藏状态，运行时间有没有变化？
* 将本节中使用的激活函数替换成ReLU，重复本节的实验。



## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/989)

![](../img/qr_rnn-scratch.svg)